In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
# 신경망 레이어
n_hidden = 256
n_input = 28 * 28
n_noise = 128 # 생성기의 입력값으로 사용할 노이즈 크기



In [4]:
## 신경망 모델 구성

X = tf.placeholder(tf.float32, [None, n_input])

# 노이즈 Z를 입력값으로 사용한다.
Z = tf.placeholder(tf.float32, [None, n_noise])

# 생성기 신경망에 사용하는 변수들이다.
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 판별기 신경망에 사용하는 변수들이다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
# 판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라값이다.
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))


In [5]:
# 생성기(G) 신경망 구성

def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

# 판별기(D) 신경망 구성

def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

# 랜덤한 노이즈 (Z)를 만든다.

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [6]:
# 노이즈를 이용해 랜덤한 이미지 생성.
G = generator(Z)
# 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구한다.
D_gene = discriminator(G)
# 진짜 이미지를 이용해 판별한 값을 구한다.
D_real = discriminator(X)

In [7]:
# 논문에 따르면, GAN 모델의 최적화는 loss_G 와 loss_D 를 최대화 하는 것 입니다.
# 다만 loss_D와 loss_G는 서로 연관관계가 있기 때문에 두 개의 손실값이 항상 같이 증가하는 경향을 보이지는 않을 것 입니다.
# loss_D가 증가하려면 loss_G는 하락해야하고, loss_G가 증가하려면 loss_D는 하락해야하는 경쟁관계에 있기 때문입니다.
# 논문의 수식에 따른 다음 로직을 보면 loss_D 를 최대화하기 위해서는 D_gene 값을 최소화하게 됩니다.
# 판별기에 진짜 이미지를 넣었을 때에도 최대값을 : tf.log(D_real)
# 가짜 이미지를 넣었을 때에도 최대값을 : tf.log(1 - D_gene)
# 갖도록 학습시키기 때문입니다.
# 이것은 판별기는 생성기가 만들어낸 이미지가 가짜라고 판단하도록 판별기 신경망을 학습시킵니다.
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
# 반면 loss_G 를 최대화하기 위해서는 D_gene 값을 최대화하게 되는데,
# 이것은 가짜 이미지를 넣었을 때, 판별기가 최대한 실제 이미지라고 판단하도록 생성기 신경망을 학습시킵니다.
# 논문에서는 loss_D 와 같은 수식으로 최소화 하는 생성기를 찾지만,
# 결국 D_gene 값을 최대화하는 것이므로 다음과 같이 사용할 수 있습니다.
loss_G = tf.reduce_mean(tf.log(D_gene))

In [8]:
# loss_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# loss_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용하여 최적화를 합니다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [14]:
# GAN 논문의 수식에 따르면 loss 를 극대화 해야하지만, minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                         var_list=G_var_list)

In [16]:
### 모델 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D , loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        # 판별기와 생성기 신경망을 각각 학습시킨다.
        _, loss_val_D = sess.run([train_D, loss_D], 
                                 feed_dict = {X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], 
                                 feed_dict = {Z: noise})
        
    print('Epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    
    ### 학습이 이뤄지는거 확인하기위해서 이미지 생성
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Z: noise})
        
        fig, ax = plt.subplots(1, sample_size , figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
        
print('끝!')       

Epoch: 0000 D loss: -0.5166 G loss: -1.973
Epoch: 0001 D loss: -0.4383 G loss: -2.203
Epoch: 0002 D loss: -0.2048 G loss: -2.714
Epoch: 0003 D loss: -0.4271 G loss: -1.631
Epoch: 0004 D loss: -0.2387 G loss: -2.22
Epoch: 0005 D loss: -0.4342 G loss: -2.46
Epoch: 0006 D loss: -0.4569 G loss: -2.422
Epoch: 0007 D loss: -0.2999 G loss: -2.565
Epoch: 0008 D loss: -0.2444 G loss: -2.72
Epoch: 0009 D loss: -0.2964 G loss: -2.723
Epoch: 0010 D loss: -0.4695 G loss: -2.236
Epoch: 0011 D loss: -0.351 G loss: -2.833
Epoch: 0012 D loss: -0.4828 G loss: -2.284
Epoch: 0013 D loss: -0.3721 G loss: -2.422
Epoch: 0014 D loss: -0.3817 G loss: -2.415
Epoch: 0015 D loss: -0.284 G loss: -2.614
Epoch: 0016 D loss: -0.5508 G loss: -2.275
Epoch: 0017 D loss: -0.423 G loss: -2.484
Epoch: 0018 D loss: -0.3614 G loss: -2.464
Epoch: 0019 D loss: -0.3128 G loss: -2.332
Epoch: 0020 D loss: -0.3742 G loss: -2.732
Epoch: 0021 D loss: -0.4251 G loss: -2.582
Epoch: 0022 D loss: -0.3565 G loss: -2.486
Epoch: 0023 D los